In [7]:
#   Dependencies
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float, DateTime


---
# Database Creation

### Mock Data

In [28]:
#   Read in Mock data
df = pd.read_csv("./../results_with_sentiment.csv")

#   Display preview
df.head(2)

,Id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,Review,Liked,sentiment,Cuisine,Restaurant_id
0,0,0.000,0.435,0.565,0.5994,0.002303,0.010709,0.986987,Wow... Loved this place.,1,5 stars,Italian,0
1,1,0.445,0.555,0.000,-0.3412,0.964183,0.032228,0.003589,Crust is not good.,0,2 stars,Italian,0


In [29]:
#   Display columns
df.columns

Index(['Id', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound',
       'roberta_neg', 'roberta_neu', 'roberta_pos', 'Review', 'Liked',
       'sentiment', 'Cuisine', 'Restaurant_id'],
      dtype='object')

In [37]:
#   Establish Base for table's class construction
Base = declarative_base()

#   mock table class construction
class Mock(Base):
    __tablename__ = "mock"

    id = Column(Integer, primary_key = True)

    vader_neg = Column(Float)
    vader_neu = Column(Float)
    vader_pos = Column(Float)
    vader_compound = Column(Float)

    roberta_neg = Column(Float)
    roberta_neu = Column(Float)
    roberta_pos = Column(Float)
    
    review = Column(String)
    liked = Column(Integer)
    sentiment = Column(String)

    cuisine = Column(String)
    restaurant_id = Column(Integer)

In [40]:
#   Set index to prepare for sql exportation
df.set_index("Id", inplace = True)

In [41]:
#   Create engine 
engine = create_engine("sqlite:///../reviews.sqlite")

#   Connect to database
con = engine.connect()

#   Create the mock table within the db
Base.metadata.create_all(con)

#   Export df to db table
df.to_sql("mock", con, if_exists = "append")

#### some data exploration

In [6]:
df.Cuisine.unique()

array(['Italian', 'Chinese', 'Indian', 'Mexican', 'Japanese',
       'Mediterranean', 'Burgers', 'Thai'], dtype=object)

### Connection test

In [42]:
from sqlalchemy.ext.automap import automap_base

#   reviews.db setup
engine = create_engine("sqlite:///../reviews.sqlite")
Base = automap_base()
Base.prepare(autoload_with = engine, reflect = True)

#   Store tables into variables
mock = Base.classes.mock

In [59]:
#   Create Session
from sqlalchemy import func

session = Session(engine)

query = session.query(mock.restaurant_id,func.avg(mock.vader_compound)).group_by(mock.restaurant_id)

testdictlist = []


for row in query:
    
    testdict = {}
    
    testdict["id"] = row[0]
    testdict["compound_average"] = row[1]

    testdictlist.append(testdict)

testdictlist




[{'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878},
 {'id': 14, 'compound_average': 0.21946836734693878}]

## Lorem Data